#### 加载所需的包

In [ ]:
from sklearn_relief import RReliefF
from pandas import read_csv
import numpy as np
import pandas as pd
import geatpy as ea
import sklearn
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV 

## Q2

### Filter阶段

In [8]:
#定义MinMax标准化函数
def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

#读取数据
dataset = read_csv('Data-368.csv', encoding='utf-8')
normlized_data = normalization(dataset)
input_matrix = np.asmatrix(normlized_data.iloc[:,1:])
label_vector = normlized_data.iloc[:,0]

#RReliefF算法处理
R = RReliefF()
feature_weights = R.fit(input_matrix, label_vector)

#权重的标准化
normlized_feature_weights = normalization(feature_weights.w_)
normlized_feature_weights = normlized_feature_weights / np.sum(normlized_feature_weights)

#处理结果导出
weights_df = pd.DataFrame(normlized_feature_weights)
writer = pd.ExcelWriter('权重及归一化数据.xlsx')
weights_df.to_excel(writer,"权重")
normlized_data.to_excel(writer,"归一化数据")
writer.save()

### Wrapper阶段

In [ ]:
#读取数据
Normalized_Data_After_Filter = read_csv('Data-after-Filter.csv')
Y = Normalized_Data_After_Filter.iloc[:, 1]
X = Normalized_Data_After_Filter.iloc[:, 2:]

#### MyProblem

In [ ]:
class MyProblem(ea.Problem): # 继承Problem父类
    def __init__(self):
        name = 'MyProblem' # 初始化name（函数名称，可以随意设置）
        M = 1 # 初始化M（目标维数）
        maxormins = [-1] # 初始化maxormins（目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标）
        Dim = 211 # 初始化Dim（决策变量维数）
        varTypes = [1] * Dim # 初始化varTypes（决策变量的类型，元素为0表示对应的变量是连续的；1表示是离散的）
        lb = [0] * Dim # 决策变量下界
        ub = [1] * Dim # 决策变量上界
        lbin = [1] * Dim # 决策变量下边界（0表示不包含该变量的下边界，1表示包含）
        ubin = [1] * Dim # 决策变量上边界（0表示不包含该变量的上边界，1表示包含）
        # 调用父类构造方法完成实例化
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)
        
        
    def aimFunc(self, pop): # 目标函数
        x = pop.Phen # 得到决策变量矩阵
        NIND = x.shape[0]
        alpha = 0.5
        #SVR
        """grid = GridSearchCV(SVR(), param_grid={"C":[100], "gamma":['scale']}, cv=10)  
        grid.fit(X, Y) 
        best_params_0 = grid.best_params_"""
        clf = SVR(kernel='rbf', C=100, gamma='scale')
        mse_0 = -cross_val_score(clf, X, Y, cv=10, scoring='neg_mean_squared_error')
        m_0 = np.mean(mse_0)
        f_0 = X.shape[1] 
        


        ObjV = []
        for i in range(NIND):
            feature_index = np.where(x[[i],]==1)[1]
            feature_selection = X.iloc[:, feature_index]
            """grid = GridSearchCV(SVR(), param_grid={"C":[1, 10, 100, 1000, 5000], "gamma":['auto', 1e-4, 1e-3, 0.01, 0.1, 1]}, cv=10)  
            grid.fit(feature_selection, Y) 
            best_params = grid.best_params_"""
            clf = SVR(kernel='rbf', C=100, gamma='scale')
            mse = -cross_val_score(clf, feature_selection, Y, cv=10, scoring='neg_mean_squared_error')
            m_s = np.mean(mse)
            f_s = np.sum(x[i])
            obj_func = alpha*(f_0 - f_s) / f_0 + (m_0 - m_s) / m_0
            ObjV.append(obj_func)
        pop.ObjV = np.array([ObjV]).T # 计算目标函数值，赋值给pop种群对象的ObjV属性 

#### Main

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import geatpy as ea # import geatpy
#from MyProblem import MyProblem # 导入自定义问题接口

if __name__ == '__main__':
    """===============================实例化问题对象==========================="""
    problem = MyProblem() # 生成问题对象
    """=================================种群设置==============================="""
    Encoding = 'BG'       # 编码方式
    NIND = 200             # 种群规模
    Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders) # 创建区域描述器
    population = ea.Population(Encoding, Field, NIND) # 实例化种群对象（此时种群还没被初始化，仅仅是完成种群对象的实例化）
    """===============================算法参数设置============================="""
    myAlgorithm = ea.soea_SEGA_templet(problem, population) # 实例化一个算法模板对象
    myAlgorithm.MAXGEN = 100 # 最大进化代数
    """==========================调用算法模板进行种群进化======================="""
    [population, obj_trace, var_trace] = myAlgorithm.run() # 执行算法模板
    population.save() # 把最后一代种群的信息保存到文件中
    # 输出结果
    best_gen = np.argmin(problem.maxormins * obj_trace[:, 1]) # 记录最优种群个体是在哪一代
    best_ObjV = obj_trace[best_gen, 1]
    print('最优的目标函数值为：%s'%(best_ObjV))
    print('最优的控制变量值为：')
    for i in range(var_trace.shape[1]):
        print(var_trace[best_gen, i])
    print('有效进化代数：%s'%(obj_trace.shape[0]))
    print('最优的一代是第 %s 代'%(best_gen + 1))
    print('评价次数：%s'%(myAlgorithm.evalsNum))
    print('时间已过 %s 秒'%(myAlgorithm.passTime))

## Q3

In [ ]:
Normalized_Data_After_Filter_Wrapper = read_csv('Data-after-Filter-Wrapper.csv',encoding='latin-1')
Y = Normalized_Data_After_Filter_Wrapper.iloc[:, 0]
X = Normalized_Data_After_Filter_Wrapper.iloc[:, 1:]

#十折交叉验证加网格搜索调参
grid = GridSearchCV(SVR(), param_grid={"C":[1, 5, 10, 50, 100, 500, 1000], "gamma":['scale', 0.1, 0.01, 1e-03, 1e-04]}, cv=10)  
grid.fit(X, Y) 
best_params = grid.best_params_

clf = SVR(kernel='rbf', **best_params)
mse = -cross_val_score(clf, X, Y, cv=10, scoring='neg_mean_squared_error')
m_ave = np.mean(mse)

## Q4

#### 建立产品S含量的预测模型

In [ ]:
S_Modeling_Data = read_csv('S-Modelin-Data.csv')
S_Y = S_Modeling_Data.iloc[:, 0]
S_X = S_Modeling_Data.iloc[:, 3:]

S_Modeling_grid = GridSearchCV(SVR(), param_grid={"C":[1, 5, 10, 50, 100, 500, 1000], "gamma":['scale', 1, 0.1, 0.01, 1e-03, 1e-04]}, cv=10)  
S_Modeling_grid.fit(S_X, S_Y) 
S_best_params = S_Modeling_grid.best_params_

S_clf = SVR(kernel='rbf', **S_best_params)
S_mse = -cross_val_score(S_clf, S_X, S_Y, cv=10, scoring='neg_mean_squared_error')
S_m_ave = np.mean(S_mse)

S_Model = S_clf.fit(S_X, S_Y)
S_Model.predict(np.array(S_X.iloc[0,:]).reshape(1, -1))[0]

#### 读取数据

In [ ]:
Normalized_Data_After_Filter_Wrapper = read_csv('Data-after-Filter-Wrapper.csv',encoding='latin-1')
Y = Normalized_Data_After_Filter_Wrapper.iloc[:, 0]
X = Normalized_Data_After_Filter_Wrapper.iloc[:, 1:]
"""index = np.where(np.array(Y) != 0)[0]#Y取最小值时候归一化后为0
Y = Y[index]
X = X.iloc[index,:]"""
append_data = X.iloc[:, :2]

#### Optimization_My_Problem

In [ ]:
class Optimization_MyProblem(ea.Problem): # 继承Problem父类
    def __init__(self):
        name = 'Optimization_MyProblem' # 初始化name（函数名称，可以随意设置）
        M = 1 # 初始化M（目标维数）
        maxormins = [-1] # 初始化maxormins（目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标）
        Dim = 19 # 初始化Dim（决策变量维数）
        varTypes = [0] * Dim # 初始化varTypes（决策变量的类型，元素为0表示对应的变量是连续的；1表示是离散的）
        lb = [0] * Dim # 决策变量下界
        ub = [1] * Dim # 决策变量上界
        lbin = [1] * Dim # 决策变量下边界（0表示不包含该变量的下边界，1表示包含）
        ubin = [1] * Dim # 决策变量上边界（0表示不包含该变量的上边界，1表示包含）
        # 调用父类构造方法完成实例化
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)
        
        
    def aimFunc(self, pop): # 目标函数
        Vars = pop.Phen # 得到决策变量矩阵
        NIND = Vars.shape[0]
        clf = SVR(kernel='rbf', **best_params)
        forecast_model = clf.fit(X, Y)
        S_clf = SVR(kernel='rbf', **S_best_params)
        S_Model = S_clf.fit(S_X, S_Y)
        
        ObjV = []
        for i in range(NIND):
            train_Vars = np.hstack((append_data_order, Vars[i]))
            loss_predict = forecast_model.predict(np.array(train_Vars).reshape(1, -1))[0]    
            reversed_loss_predict = loss_predict * 1.62 + 0.2 #需要调整回原来的数据维度
            reversed_loss = (train_Vars[0] * 6.4 + 85.3) - (train_Vars[1] * 5.32 + 85.1)
            obj_func = 1 - reversed_loss_predict / reversed_loss
            ObjV.append(obj_func)
        pop.ObjV = np.array([ObjV]).T # 计算目标函数值，赋值给pop种群对象的ObjV属性   
        # 采用可行性法则处理约束
        pop.CV = S_Model.predict(Vars).reshape(-1, 1)- (5-3.2)/8.6
    def calReferObjV(self): # 设定目标数参考值0.3
        referenceObjV = np.array([[0.3]])
        return referenceObjV


#### Optimization_Main

In [ ]:
if __name__ == '__main__':
    """================================实例化问题对象==========================="""
    for sample_num in range(len(append_data)):
        try:
            print("样本" + str(sample_num))
            append_data_order = np.array(append_data.iloc[sample_num,:])
            problem = Optimization_MyProblem() # 生成问题对象
            """==================================种群设置==============================="""
            Encoding = 'RI'       # 编码方式
            NIND = 1000            # 种群规模
            Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders) # 创建区域描述器
            population = ea.Population(Encoding, Field, NIND) # 实例化种群对象（此时种群还没被初始化，仅仅是完成种群对象的实例化）
            """================================算法参数设置============================="""
            myAlgorithm = ea.soea_DE_rand_1_bin_templet(problem, population) # 实例化一个算法模板对象
            myAlgorithm.MAXGEN = 500 # 最大进化代数
            myAlgorithm.mutOper.F = 0.5 # 差分进化中的参数F
            myAlgorithm.recOper.XOVR = 0.7 # 重组概率




            """===========================调用算法模板进行种群进化======================="""
            [population, obj_trace, var_trace] = myAlgorithm.run() # 执行算法模板
            population.save() # 把最后一代种群的信息保存到文件中
            # 输出结果
            best_gen = np.argmin(problem.maxormins * obj_trace[:, 1]) # 记录最优种群个体是在哪一代
            best_ObjV = obj_trace[best_gen, 1]
            print('最优的目标函数值为：%s'%(best_ObjV))
            print('最优的决策变量值为：')
            for i in range(var_trace.shape[1]):
                print(var_trace[best_gen, i])
            print('有效进化代数：%s'%(obj_trace.shape[0]))
            print('最优的一代是第 %s 代'%(best_gen + 1))
            print('评价次数：%s'%(myAlgorithm.evalsNum))
            print('时间已过 %s 秒'%(myAlgorithm.passTime))
            print("")
        except RuntimeError:
            print("样本" + str(sample_num) + "没找到可行解。")
            print("")